# Cypher Generation

generate cypher to retrieve data in same age group/income group

In [2]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import re

In [3]:
person_schema = """
**Person**
- 'income': INTEGER Min: -1616, Max: 927313 description: Individual annual income of the person in USD.
- 'age': INTEGER Min: 17, Max: 94 description: Age of the person.
"""

# - 'vehicles': STRING Available options: ['1', '0', '2', '3_plus'] description: Number of vehicles available to the person.
# - 'family_structure': STRING Available options: ['family_single', 'married_couple', 'nonfamily_single', 'living_alone'] description: Family structure of the person.
# - 'household_size': STRING Available options: ['4_person', '5_person', '3_person', '2_person', '1_person', '7_plus_person', '1_person_group_quarters', '6_person'] description: Number of people in the household this person belongs to.


cypher_template = """Based on the Neo4j graph schema below, complete the Cypher query to find the {number} nodes similar to the profile:
{schema}
Profile: {profile}

Groups:
- 'income' : 
  - 'Debt' : <0
  - 'Low' : ≥0 AND <30000
  - 'Moderate' : ≥30000 AND <60000
  - 'High' : ≥60000 AND <90000
  - 'Very High' : ≥90000 AND <120000
  - 'Ultra High' : ≥120000

- 'age' : 
  - 'Teen' : <18
  - 'Young Adult' : ≥18 AND <30
  - 'Adult' : ≥30 AND <40
  - 'Middle Age' : ≥40 AND <50
  - 'Senior' : ≥50 AND <60
  - 'Elderly' : ≥60

**IMPORTANT:**
- **DO NOT** include other information except age and income.
- **DO NOT** include queries for filters not mentioned in the profile. For example, if the profile is "a young man" and there is no information about income, **DO NOT** generate a query for income.
- Map the relevant age or income information into these groups. For example, "a young man" should translate to `p.age ≥18 AND p.age<30` for age.
- Keep other parts of the query as is, and answer with the whole query.
- Wrap the final answer in triple backticks (```cypher ```) to indicate a code block.

Answer: 
age group: e.g. 'High'
income group: e.g. 'Moderate'
```cypher
MATCH (p:Person)
WHERE [[your response query here]]
WITH p ORDER BY rand() LIMIT {number}
MATCH (p)-[r1:WANT_TO]-(d:Desire {{desire:'{desire}'}})-[r2:GO_TO]-(i:Intention)
RETURN COLLECT(DISTINCT p) AS person, 
    COLLECT(DISTINCT d) AS desire, 
    COLLECT(DISTINCT i) AS intention,
    COLLECT(DISTINCT r1) AS want_to,
    COLLECT(DISTINCT r2) AS go_to
```
"""


In [4]:
class CypherOutputParser(StrOutputParser):
    def parse(self, text: str) -> str:
        code_pattern = re.compile(r'```(?:cypher)?(.*?)```', re.DOTALL)
        match = code_pattern.search(text)
        if match:
            return match.group(1).strip()
        return "No cypher found"

In [5]:
cypher_model = ChatOllama(model='llama3.1',temperature=0)
cypher_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input query, convert it to a Cypher query. No pre-amble.",
        ),
        ("human", cypher_template),
    ]
)
cypher_response = (
    RunnablePassthrough.assign(
        schema=lambda _: person_schema,
    )
    | cypher_prompt
    | cypher_model.bind()
    # | CypherOutputParser()
    | StrOutputParser()
)

In [7]:
profile = 'A Middle Age person, whose income is Very High'
top_k = 50
desire = 'Eat'

cypher = cypher_response.invoke({
    "profile": profile,
    "number": top_k,
    "desire": desire
})
print(cypher)

Based on the profile "A Middle Age person, whose income is Very High", we can map this information into the following groups:

- Age group: 'Middle Age'
- Income group: 'Very High'

Here's the completed Cypher query:
```cypher
MATCH (p:Person)
WHERE p.age >= 40 AND p.age < 50 AND p.income >= 90000 AND p.income < 120000
WITH p ORDER BY rand() LIMIT 50
MATCH (p)-[r1:WANT_TO]-(d:Desire {desire:'Eat'})-[r2:GO_TO]-(i:Intention)
RETURN COLLECT(DISTINCT p) AS person, 
    COLLECT(DISTINCT d) AS desire, 
    COLLECT(DISTINCT i) AS intention,
    COLLECT(DISTINCT r1) AS want_to,
    COLLECT(DISTINCT r2) AS go_to
```


In [6]:
url="bolt://localhost:7687"
username="neo4j"
password="neo4jgraph"
embedding_model = OllamaEmbeddings(model="nomic-embed-text")

graph = Neo4jGraph (
    url=url,
    username=username,
    password=password,
    enhanced_schema=True,
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [7]:
results = graph.query(cypher)
print(len(results[0]['person']))

8
